## HotChili Analytics trading notebook template
#### Configure by setting ALGO_NAME in cell below.
#### Run various options (backtest, ingest, live) by uncommenting one cell 

In [1]:
%matplotlib inline
%load_ext zipline

# %reload_ext zipline # Uncomment and use this when already loaded zipline extension for magic cell usage.

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [2]:
import pandas as pd

# Options you can uncomment and set:

# pd.set_option("max_colwidth", 300)
# pd.set_option("display.max_rows", 300)
# pd.set_option("display.max_columns", 50)
# pd.set_option('precision', 2)
# pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
import os

hca_root_path = os.environ['HCA_ROOT']
print(f"hca_root_path = {hca_root_path}")

hca_root_path = /home/hca-ws2004/hca


# Construct algorithm strategy path names

Assumptions:

- the strategy is in a directory with the same name as the strategy in `ALGO_NAME` below
- the strategy is located in the hca-resources directory, which is located relative to `hca_root_path`, found above

In [4]:
ALGO_NAME = "HCA_In_and_Out" # <--- Supply name here

Other variables are derived from `ALGO_NAME`:

In [5]:
HCA_RESOURCES_PATH = hca_root_path + "/hca-resources/" 
ALGO_PATH          = HCA_RESOURCES_PATH + ALGO_NAME + "/" 
ALGO_BT            = ALGO_PATH + ALGO_NAME + ".py"
ALGO_BT_OUT        = ALGO_PATH + ALGO_NAME + ".pkl"
ALGO_LIVE          = ALGO_PATH + ALGO_NAME + "_Live" + ".py"

print(f"""
ALGO_NAME          = {ALGO_NAME}
HCA_RESOURCES_PATH = {HCA_RESOURCES_PATH}
ALGO_PATH          = {ALGO_PATH}
ALGO_BT            = {ALGO_BT}
ALGO_BT_OUT        = {ALGO_BT_OUT}
ALGO_LIVE          = {ALGO_LIVE}

Contents of algo directory:
""")

!ls $ALGO_PATH


ALGO_NAME          = HCA_In_and_Out
HCA_RESOURCES_PATH = /home/hca-ws2004/hca/hca-resources/
ALGO_PATH          = /home/hca-ws2004/hca/hca-resources/HCA_In_and_Out/
ALGO_BT            = /home/hca-ws2004/hca/hca-resources/HCA_In_and_Out/HCA_In_and_Out.py
ALGO_BT_OUT        = /home/hca-ws2004/hca/hca-resources/HCA_In_and_Out/HCA_In_and_Out.pkl
ALGO_LIVE          = /home/hca-ws2004/hca/hca-resources/HCA_In_and_Out/HCA_In_and_Out_Live.py

Contents of algo directory:

HCA_In_and_Out.ipynb		  HCA_In_and_Out_tearsheet.html
HCA_In_and_Out_Live.py		  HCA_In_and_Out_tearsheet.ipynb
HCA_In_and_Out.pkl		  ib-live-in_and_out-2021-05-27.log
HCA_In_and_Out.py		  Q_In_and_Out_orig.py
HCA_In_and_Out_Q.py		  strategy.state
HCA_In_and_Out_tearsheet(3).html


## Zipline backtest: 

- Method: Jupyter magic cell (%%) 
- Execution of zipline code in cell containing command line command
- Uncomment first line and hit (shift-enter) inside the cell to run simulation backtest

In [6]:
%%zipline --start=2020-1-1 --end=2021-5-26 -b sharadar-eqfd -o $ALGO_BT_OUT

# Vlad Code from: Aleksei Dremov  in
# https://www.quantopian.com/posts/live-slash-paper-trade-the-in-out-stragegy

# Price relative ratios (intersection) with wait days
import numpy as np

from zipline.api import order, cancel_order, get_open_orders, symbol, symbols, date_rules, time_rules, order_target_percent, record, schedule_function, get_datetime
from trading_calendars import get_calendar

def initialize(context):
    # -----------------------------------------------------------------------------------------------
    # YAHOO_SYM_LST=QQQ,TLT,IEF,SLV,GLD,XLI,XLU zipline ingest -b yahoo_direct
    c = context
    #orig c.STOCKS = symbols('QQQ'); c.BONDS = symbols('TLT','IEF'); c.LEV = 1.00; c.wt = {};
    #ajjc
    c.STOCKS = symbols('QQQ', 'URTH'); c.BONDS = symbols('TLT','IEF','SCHD'); c.LEV = 1.00; c.wt = {};
    c.A = symbol('SLV'); c.B = symbol('GLD'); c.C = symbol('XLI'); c.D = symbol('XLU');
    c.MKT = symbol('QQQ'); c.VOLA = 126; c.LB = 1.00; c.BULL = 1; c.COUNT = 0; c.OUT_DAY = 0; c.RET_INITIAL = 80;
# -----------------------------------------------------------------------------------------------

    # schedule_function(daily_check, date_rules.every_day(), time_rules.market_open(minutes = 140))
    # schedule_function(record_vars, date_rules.every_day(), time_rules.market_close())
    
def minut(context): #Minute of trading day
    dt = context.get_datetime().astimezone(pytz.timezone('US/Eastern'))
    return (dt.hour * 60) + dt.minute - 570
            
#def handle_data(context, data):
#    time_now = minut(context)
#    log.info("___handle_data: {} = Current Trading Minute".format(time_now))
     
def handle_data(context, data):
    if (not context.ORDERS_DONE):
        context.ORDERS_DONE = True
        daily_check(context,data) #Name of the original algo trading function
        trade(context,data,context.wt)
        record_vars(context, data)        
    else:
        print("Exiting: zipline-broker: context.portfolio : {}".format(context.portfolio))
        exit()
        
def before_trading_start(context, data):
    c = context
    c.ORDERS_DONE       = False #No Orders done yet today
    c.all_orders = {} 
# End:Zipline Builtin Functions
        

def daily_check(context,data):
    c = context
    #global c.BULL, c.COUNT, OUT_DAY
    vola = data.history(c.MKT, 'price',  c.VOLA + 1, '1d').pct_change().std() * np.sqrt(252)
    WAIT_DAYS = int(vola * c.RET_INITIAL)
    RET = int((1.0 - vola) * c.RET_INITIAL)
    P = data.history([c.A,c.B,c.C,c.D], 'price',  RET + 2, '1d').iloc[:-1].dropna()
    ratio_ab = (P[c.A].iloc[-1] / P[c.A].iloc[0]) / (P[c.B].iloc[-1] / P[c.B].iloc[0])
    ratio_cd = (P[c.C].iloc[-1] / P[c.C].iloc[0]) / (P[c.D].iloc[-1] / P[c.D].iloc[0])
    exit = ratio_ab < c.LB and ratio_cd < c.LB
    if exit: c.BULL = 0; c.OUT_DAY = c.COUNT;
    elif (c.COUNT >= c.OUT_DAY + WAIT_DAYS): c.BULL = 1
    c.COUNT += 1
    wt_stk = c.LEV if c.BULL else 0;
    wt_bnd = 0 if c.BULL else c.LEV;
    for sec in c.STOCKS: c.wt[sec] = wt_stk / len(c.STOCKS);
    for sec in c.BONDS: c.wt[sec] = wt_bnd / len(c.BONDS)
    
    print("WAIT_DAYS={} RET={} exit={} c.COUNT={} c.BULL={} c.OUT_DAY={}".format(WAIT_DAYS,RET,exit,c.COUNT,c.BULL,c.OUT_DAY))
    print("Trading Weights Today: wt_stk={} wt_bnd={} wts={}".format(c.wt,wt_stk,wt_bnd))
    
    #for sec, weight in c.wt.items():
        #order_target_percent(sec, weight)
    #record( wt_bnd = wt_bnd, wt_stk = wt_stk )

def trade(context,data,wts):
    lev = context.LEV #Should allow for a small percentage of Cash, to enable ordering fluctuations without having order cancelled.
    
    ### ajjc: Find a way to return if already traded today
    
    #print("TradingLinkData: Zipline-broker: context.portfolio : {}".format(context.portfolio))
    #print("TradingLinkData: IB-Account    : context.account   : {}".format(context.account))
    
    #acct_liq    = context.portfolio.starting_cash #Same as IB net_liquidation when measured daily
    acct_liq    = context.portfolio.portfolio_value #Same as IB net_liquidation
    acct_invest = lev * acct_liq   
    positions      = context.portfolio.positions
    #Live positions      = context.broker.positions
    
    # Get rid of positions not in wts
    for key in positions:
        if (key not in wts and not get_open_orders(key)):
            order(key, -positions[key].amount)      
        
    for stk,weight in wts.items():
        if data.can_trade(stk) and not get_open_orders(stk):
            if stk in positions:
                current_amt = positions[stk].amount
                rebalance_amt = int(acct_invest*(weight) / data.current(stk,'price'))
                delta_amt = rebalance_amt - current_amt
                if delta_amt != 0:
                    order(stk, delta_amt) 
                else:
                    print("No new orders for : {}".format(stk))

            if (stk not in positions and data.can_trade(stk) 
                and not get_open_orders(stk)):
                
                amt = int(acct_invest*(weight) / data.current(stk,'price'))
                order(stk, amt)                                                    

def record_vars(context, data):
    record(leverage = context.account.leverage)

extension: hca_root_path = /home/hca-ws2004/hca
extension:TODAY_STR = 2021-05-27
extension:TWO_YR_AGO_STR = 2019-05-27
extension:  start_date=2019-05-27 end_date = 2021-05-27
WAIT_DAYS=12 RET=67 exit=False c.COUNT=1 c.BULL=1 c.OUT_DAY=0
Trading Weights Today: wt_stk={Equity(13117 [QQQ]): 0.5, Equity(16277 [URTH]): 0.5, Equity(15654 [TLT]): 0.0, Equity(8108 [IEF]): 0.0, Equity(14005 [SCHD]): 0.0} wt_bnd=1.0 wts=0
TradingLinkData: Zipline-broker: context.portfolio : Portfolio({'cash_flow': 0.0, 'starting_cash': 10000000.0, 'portfolio_value': 10000000.0, 'pnl': 0.0, 'returns': 0.0, 'cash': 10000000.0, 'positions': {}, 'start_date': Timestamp('2020-01-02 00:00:00+0000', tz='UTC', freq='C'), 'positions_value': 0.0, 'positions_exposure': 0.0})
TradingLinkData: IB-Account    : context.account   : Account({'settled_cash': 10000000.0, 'accrued_interest': 0.0, 'buying_power': inf, 'equity_with_loan': 10000000.0, 'total_positions_value': 0.0, 'total_positions_exposure': 0.0, 'regt_equity': 100000

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2020-01-02 21:00:00+00:00,NaN,0.000000,NaN,0.009352,NaN,NaN,0.000000e+00,1.000000e+07,0.000000e+00,0.000000e+00,...,0.0,0.0,0,NaN,1.000000e+07,0.000000e+00,0.000000e+00,1,[],0.0
2020-01-03 21:00:00+00:00,0.003090,-0.000275,-0.037610,0.001709,0.189973,0.016265,-5.451979e+06,4.548021e+06,5.449227e+06,5.449227e+06,...,0.0,0.0,0,-11.224972,1.000000e+07,0.000000e+00,0.000000e+00,2,"[{'amount': 23131, 'dt': 2020-01-03 21:00:00+0...",0.0
2020-01-06 21:00:00+00:00,0.032339,0.003107,0.261002,0.005530,0.136981,0.061743,-1.787965e+06,2.760055e+06,7.271010e+06,7.271010e+06,...,0.0,0.0,0,103.462366,4.548021e+06,5.449227e+06,5.449227e+06,3,"[{'amount': 17556, 'dt': 2020-01-06 21:00:00+0...",0.0
2020-01-07 21:00:00+00:00,0.029849,0.002386,0.143653,0.002703,0.117843,0.092827,-1.212015e+06,1.548041e+06,8.475823e+06,8.475823e+06,...,0.0,0.0,0,24.664498,2.760055e+06,7.271010e+06,7.271010e+06,4,"[{'amount': 12392, 'dt': 2020-01-07 21:00:00+0...",0.0
2020-01-08 21:00:00+00:00,0.039035,0.007115,0.335653,0.008047,0.107226,0.167777,-3.790479e+05,1.168993e+06,8.902157e+06,8.902157e+06,...,0.0,0.0,0,65.612695,1.548041e+06,8.475823e+06,8.475823e+06,5,"[{'amount': 3847, 'dt': 2020-01-08 21:00:00+00...",0.0
2020-01-09 21:00:00+00:00,0.047480,0.013547,0.503185,0.014882,0.101565,0.253877,-1.127245e+06,4.174711e+04,1.009372e+07,1.009372e+07,...,0.0,0.0,0,113.720389,1.168993e+06,8.902157e+06,8.902157e+06,6,"[{'amount': 11425, 'dt': 2020-01-09 21:00:00+0...",0.0
2020-01-10 21:00:00+00:00,0.054534,0.010236,0.248851,0.011962,0.098135,0.337129,-4.145481e+04,2.923076e+02,1.010206e+07,1.010206e+07,...,0.0,0.0,0,18.281929,4.174711e+04,1.009372e+07,1.009372e+07,7,"[{'amount': -48, 'dt': 2020-01-10 21:00:00+00:...",0.0
2020-01-13 21:00:00+00:00,0.065830,0.019315,0.408900,0.018921,0.095361,0.440581,-5.072460e+01,2.415830e+02,1.019291e+07,1.019291e+07,...,0.0,0.0,0,32.130921,2.923076e+02,1.010206e+07,1.010206e+07,8,"[{'amount': -17, 'dt': 2020-01-13 21:00:00+00:...",0.0
2020-01-14 21:00:00+00:00,0.067826,0.016286,0.242737,0.017368,0.091541,0.488122,-1.328916e+02,1.086915e+02,1.016275e+07,1.016275e+07,...,0.0,0.0,0,19.172955,2.415830e+02,1.019291e+07,1.019291e+07,9,"[{'amount': -58, 'dt': 2020-01-14 21:00:00+00:...",0.0
2020-01-15 21:00:00+00:00,0.064106,0.017206,0.210698,0.019667,0.086322,0.486939,-6.801546e+01,4.067601e+01,1.017202e+07,1.017202e+07,...,0.0,0.0,0,19.202850,1.086915e+02,1.016275e+07,1.016275e+07,10,"[{'amount': 23, 'dt': 2020-01-15 21:00:00+00:0...",0.0


## Display your current bundles

In [7]:
#!zipline bundles # Finds all bundles

## Ingest Sharadar funds assets for today

In [8]:
# Ingest Sharadar funds assets for today, if needed.

# Only need to ingest Funds for this algo, and this takes less processing time and system memory than ingesting
# all of Sharadar Equities plus Funds bundle (sharadar-eqfd)
# !zipline ingest -b sharadar-funds

## Zipline backtest, alternative method

- Method: command line
- Execution of zipline code, located in a file, using below command line execution with magic (`!`) invocation
- This line can also be run in a terminal by copying everything past the `!` and pasting (shift-insert) it into the target terminal

In [ ]:
#!zipline run -f $ALGO_BT  --start=2018-1-1 --end=2021-05-26 --capital-base 100000 -b sharadar-funds -o $ALGO_BT_OUT

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws2004/hca
extension:TODAY_STR = 2021-05-27
extension:TWO_YR_AGO_STR = 2019-05-27
extension:  start_date=2019-05-27 end_date = 2021-05-27
[2021-05-27 20:04:22.040702] WARNING: Loader: Refusing to download new benchmark data because a download succeeded at 2021-05-27 19:28:13.938266+00:00.
[2021-05-27 20:04:22.096940] WARNING: Loade

### Run Zipline live on IB-TWS via command line

- Method: command line
- Execution of zipline code using below command line execution using magic (`!`) invocation

**Notes:** 
- IB-TWS or IB-Gateway must be running, with `IB_ACCT` and `IB_URI` port being correct to live trade
- Change `I_WANT_TO_RUN_THIS_CODE` to `True` below to run zipline live on IB-TWS/IB-Gateway

In [ ]:
TODAY = pd.datetime.today().strftime("%Y-%m-%d")
print("TODAY = {}".format(TODAY))

In [ ]:
ALGO_STATE = ALGO_PATH + "strategy.state" 
ALGO_RTB   = ALGO_PATH + "realtime-bars/"

# Edit the following URI to match your IB account and port info.
IB_ACCT = "DU1568488"
IB_URI = "127.0.0.1:7497:1301"

# Change following to 'True' and run cell (control-enter) to execute live run.
I_WANT_TO_RUN_THIS_CODE = False
#I_WANT_TO_RUN_THIS_CODE = True

if I_WANT_TO_RUN_THIS_CODE:
    
    !zipline run \
        -s $TODAY \
        -f $ALGO_LIVE \
        --bundle sharadar-funds \
        --broker ib \
        --broker-uri $IB_URI \
        --broker-acct $IB_ACCT \
        --data-frequency daily \
        --state-file $ALGO_STATE \
        --realtime-bar-target $ALGO_RTB 